In [ ]:
# Suppression des lignes où les deux colonnes sont NaN en même temps
data_modifiee = data_modifiee[~(data_modifiee['date_nais'].isna() & data_modifiee['date_permis'].isna())]

In [ ]:
data_modifiee = data_modifiee.dropna(subset=['date_nais', 'date_permis'])

## Création de l'identifiant unique

In [ ]:
print(dataf.columns.tolist())

# Regrouper par date de naissance et date de première souscription
groupes = dataf.groupby(['date_nais_f', 'date_permis_f']).size().reset_index(name='nb_personnes')

# Afficher les combinaisons où plusieurs personnes partagent les mêmes dates
groupes_filtrés = groupes[groupes['nb_personnes'] > 1]

print(groupes_filtrés)

In [ ]:
#Étape 2 : filtrer les lignes concernées dans le DataFrame d'origine
df_doublons = data_modifiee.merge(groupes[['date_nais_f', 'date_permis_f']], on=['date_nais_f', 'date_permis_f'])

# Étape 3 : afficher les colonnes souhaitées
colonnes = ['date_nais_f', 'date_permis_f', 'sexe', 'exercice', 'date_premsous']
df_resultat = df_doublons[colonnes]

# Affichage
print(df_resultat)


In [ ]:
import pandas as pd
import hashlib

# Étape 1 : extraire les dates brutes s’il y a du texte autour
# Format visé : "28/06/1976" ou similaire
data_modifiee['date_nais_f'] = data_modifiee['date_nais'].astype(str).str.extract(r'(\d{1,2}/\d{1,2}/\d{2,4})')[0]
data_modifiee['date_permis_f'] = data_modifiee['date_permis'].astype(str).str.extract(r'(\d{1,2}/\d{1,2}/\d{2,4})')[0]

# Étape 2 : conversion en datetime (au format français jour/mois/année)
data_modifiee['date_nais_f'] = pd.to_datetime(data_modifiee['date_nais_f'], format='%d/%m/%Y', errors='coerce')
data_modifiee['date_permis_f'] = pd.to_datetime(data_modifiee['date_permis_f'], format='%d/%m/%Y', errors='coerce')

# Étape 3 : vérifier que la conversion a bien fonctionné
print(data_modifiee['date_nais_f'].isna().sum(), "dates de naissance mal converties")
print(data_modifiee['date_permis_f'].isna().sum(), "dates de permis mal converties")

# Étape 4 : créer une chaîne concaténée propre
data_modifiee['identifiant_concat'] = (
    data_modifiee['date_nais_f'].astype(str) + '_' +
    data_modifiee['date_permis_f'].astype(str) + '_' +
    data_modifiee['sexe'].astype(str)
)

# Étape 5 : créer un identifiant unique (version hachée md5)
data_modifiee['identifiant_unique'] = data_modifiee['identifiant_concat'].apply(
    lambda x: hashlib.md5(x.encode()).hexdigest()
)

# Étape 6 (optionnel) : vérifier l’unicité
n_total = len(data_modifiee)
n_uniques = data_modifiee['identifiant_unique'].nunique()
print(f"{n_uniques} identifiants uniques sur {n_total} lignes")


In [ ]:
# Compter les occurrences de chaque triplet et trier par ordre décroissant 
compte_triplets = (
    dataf
    .groupby(['date_nais_f', 'date_permis_f', 'sexe', 'exercice'])
    .size()
    .reset_index(name='nb_apparitions')
    .sort_values(by='nb_apparitions', ascending=False)
)

print(compte_triplets)

In [ ]:
# Statistiques descriptives par exercice
stats_par_exercice = (
    compte_triplets
    .groupby('exercice')['nb_apparitions']
    .agg(['count', 'mean', 'median', 'min', 'max', 'std', 'sum', 'quantile'])
    .sort_index()
)

print(stats_par_exercice)

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

plt.figure(figsize=(12, 6))
sns.boxplot(data=compte_triplets, x='exercice', y='nb_apparitions')
plt.title('Distribution du nombre de sinistres par exercice')
plt.ylabel('Nb apparitions (sinistres)')
plt.xlabel('Exercice')
plt.xticks(rotation=45)
plt.show()

In [ ]:
# Exemple : si CRM est dans data_modifiee
df_corr = compte_triplets.merge(
    data_modifiee[['date_nais_f', 'date_permis_f', 'sexe', 'exercice', 'crm']],
    on=['date_nais_f', 'date_permis_f', 'sexe', 'exercice'],
    how='left'
)

In [ ]:
correlation = df_corr['crm'].corr(df_corr['nb_apparitions'])
print(f"Corrélation entre CRM et nombre de sinistres : {correlation:.4f}")

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

plt.figure(figsize=(8, 5))
sns.scatterplot(data=df_corr, x='nb_apparitions', y='crm', alpha=0.3)
sns.regplot(data=df_corr, x='nb_apparitions', y='crm', scatter=False, color='red', line_kws={"lw":2})
plt.title("Relation entre nombre de sinistres et CRM")
plt.xlabel("Nombre de sinistres (nb_apparitions)")
plt.ylabel("CRM")
plt.grid(True)
plt.show()

SINISTRES Par année

In [ ]:
# Créer les variables indicatrices pour les exercices 2019 à 2024
for annee in range(2019, 2025):
    data_modifiee[f'ex_{str(annee)[-2:]}'] = (data_modifiee['exercice'] == annee).astype(int)


In [ ]:
# Somme des indicateurs par identifiant_unique
somme_ex = (
    data_modifiee
    .groupby('identifiant_unique')[['ex_19', 'ex_20', 'ex_21', 'ex_22', 'ex_23', 'ex_24']]
    .sum()
    .reset_index()
)


In [ ]:
somme_ex.head()

In [ ]:
presence_par_individu = (
    data_modifiee
    .groupby('identifiant_unique')[['ex_19', 'ex_20', 'ex_21', 'ex_22', 'ex_23', 'ex_24']]
    .max()
    .reset_index()
)


In [ ]:
# Ajout d'une colonne qui compte le nombre total d'années de présence
somme_ex['total_exercices'] = (
    somme_ex[['ex_19', 'ex_20', 'ex_21', 'ex_22', 'ex_23', 'ex_24']].sum(axis=1)
)

# Tri décroissant par total_exercices
somme_ex = somme_ex.sort_values(by='total_exercices', ascending=False)

# Affichage
print(somme_ex.head())
